<h3>Neural Network from scratch</h3>

<h4> Data preprocessing </h4>

**Load the data from the final project with pandas**

In [1]:
# your code goes here
import pandas as pd
import numpy as np

data = pd.read_csv('data/pcawg_rnaseq_gene_expr_tpm.tsv.gz', compression='gzip', sep='\t', header='infer')

In [2]:
pwd

'/Users/guillemguigoicorominas/Desktop/Guillem/BDS3/BDS3-Machine-Learning/labs'

In [3]:
data = data.drop(['Symbol', 'Gene'], axis=1)

In [4]:
gene_ids = list(data.columns)

In [5]:
gene_ids = [gene.split('.')[0] for gene in gene_ids]

In [6]:
X = data.to_numpy().T

In [7]:
sample_sheet = pd.read_csv('data/pcawg_sample_sheet.tsv', sep='\t')

In [8]:
translation_dict = dict(zip(sample_sheet['icgc_sample_id'], sample_sheet['dcc_project_code']))

In [9]:
cancer_types = list(map(lambda x: translation_dict[x], gene_ids))
labels_dict = {gene_id: i for i, gene_id in enumerate(set(cancer_types))}

In [10]:
y = np.array(list(map(lambda x: labels_dict[x], cancer_types)))

**Do the train/test split with 25% of the samples in the test split. Standardize your data with the mean and standard deviation of the features of the training set**

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [12]:
X_train.shape

(1019, 19621)

In [13]:
X_test.shape

(340, 19621)

In [14]:
y_train.shape

(1019,)

In [15]:
y_test.shape

(340,)

**Store the labels in a variable called `y` and use keras `to_categorical` to convert your labels to vectors**

In [16]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

from keras.utils import to_categorical

# your code goes here
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [17]:
cancer_types_dict = {i: gene_id for i, gene_id in enumerate(set(cancer_types))}

**Use PCA to select the first 30 principal components from the 19.000+ genes. Remember that the data is in the *features x observations* format, so you need to transpose the matrix first**

In [18]:
# your code goes here
from sklearn.decomposition import PCA
pca = PCA(n_components = 30)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

<h4>Functions used by the neural network</h4>

We will now define the functions that will be used by the neural network.

The first function we need to define is the activation function sigmoid:

\begin{equation}
\sigma(x) = \frac{1}{1 + \exp{-x}}
\end{equation}

The derivative of the sigmoid is (conviniently) the following:

\begin{equation}
\frac{d\sigma(x)}{dx} = (1 - \sigma(x))·\sigma(x)
\end{equation}

And finally, the cross entropy that we will use as loss function:


In [19]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return (1-sigmoid(x)) * sigmoid(x)

def compute_loss(y_pred, y_true):
    cross_entropy = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return cross_entropy

We will now define the weight matrices of our neural network. For this particular network we will use a single hidden layer with 64 neurons and an output layer which should have as many neurons as the number of classes in our dataset. Initialize the weights with random values.

In [20]:
y_train.shape

(1019, 29)

In [21]:
import numpy as np

input_shape = X_train.shape[-1]
n_intermediate = 64
n_classes = y_train.shape[-1]

w1 = np.random.rand(input_shape, n_intermediate)
w2 = np.random.rand(n_intermediate, n_classes)
b1 = np.random.rand(n_intermediate, 1)
b2 = np.random.rand(n_classes, 1)

**Visualize the shape of your weight matrices**

In [22]:
print(w1.shape)
print(w2.shape)

(30, 64)
(64, 29)


We will use the following back propagation and forward functions:

In [23]:
def forward(X, w1, w2, b1, b2):
    return np.dot(w2.T, sigmoid(np.dot(w1.T,X.T) + b1)) + b2

**Train the neural network for 1000 epochs with a learning rate of 0.01. Print the loss at each epoch**

<h3> Neural Network with Keras </h3>

In [24]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(64, activation='sigmoid', input_shape=(X_train.shape[-1],)))
model.add(Dense(n_classes, activation='softmax'))

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1984      
                                                                 
 dense_1 (Dense)             (None, 29)                1885      
                                                                 
Total params: 3,869
Trainable params: 3,869
Non-trainable params: 0
_________________________________________________________________


In [26]:
from keras.optimizers import SGD

model.compile(
    loss="categorical_crossentropy",
    optimizer=SGD(learning_rate=0.05),
    metrics=["accuracy"]
    )

In [27]:
history = model.fit(
    X_train, 
    y_train, 
    epochs=500, 
    batch_size=50, 
    verbose=1, 
    validation_data=(X_test, y_test)
)

Epoch 1/500
21/21 [==============================] - 1s 12ms/step - loss: 3.4772 - accuracy: 0.0903 - val_loss: 3.1385 - val_accuracy: 0.1471
Epoch 2/500
21/21 [==============================] - 0s 3ms/step - loss: 2.9582 - accuracy: 0.2434 - val_loss: 2.7992 - val_accuracy: 0.2853
Epoch 3/500
21/21 [==============================] - 0s 3ms/step - loss: 2.6995 - accuracy: 0.3268 - val_loss: 2.6473 - val_accuracy: 0.3353
Epoch 4/500
21/21 [==============================] - 0s 3ms/step - loss: 2.5332 - accuracy: 0.3660 - val_loss: 2.5178 - val_accuracy: 0.3647
Epoch 5/500
21/21 [==============================] - 0s 3ms/step - loss: 2.4077 - accuracy: 0.3896 - val_loss: 2.4178 - val_accuracy: 0.3794
Epoch 6/500
21/21 [==============================] - 0s 3ms/step - loss: 2.3098 - accuracy: 0.4112 - val_loss: 2.3352 - val_accuracy: 0.3912
Epoch 7/500
21/21 [==============================] - 0s 3ms/step - loss: 2.2180 - accuracy: 0.4289 - val_loss: 2.2570 - val_accuracy: 0.4000
Epoch 8/500


KeyboardInterrupt: 

In [ ]:
def plot_loss_curves(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    epochs = range(len(history.history['loss']))

    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('loss')
    plt.xlabel('Epochs')
    plt.legend()

    plt.figure()
    plt.plot(epochs, accuracy, label='training_accuracy')
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()

In [ ]:
plot_loss_curves(history)

In [ ]:
import matplotlib.pyplot as plt